In [1]:
import numpy as np
import pandas as pd
import glob
import h5py
import os
import cv2
import math
import time
from tensorflow.keras.preprocessing.image import load_img, img_to_array

# Data type evaluation
* [numpy.iinfo()](https://numpy.org/doc/stable/reference/generated/numpy.iinfo.html)
* [numpy.finfo()](https://numpy.org/doc/stable/reference/generated/numpy.finfo.html)

In [2]:
intInfo = np.iinfo(np.uint8)
print("float info")
print("- bits\t\t", f"{intInfo.bits} bits")
print("- range\t\t",f"{intInfo.min} to {intInfo.max}")

float info
- bits		 8 bits
- range		 0 to 255


In [3]:
# ref: 
floatInfo = np.finfo(np.float16)
print("float info")
print("- bits\t\t", f"{floatInfo.bits} bits")
print("- range\t\t",f"{floatInfo.min} to {floatInfo.max}")
print("- precision\t",f"{floatInfo.precision} decimal digits "  )

float info
- bits		 16 bits
- range		 -65504.0 to 65504.0
- precision	 3 decimal digits 


# loading data into np.array 

In [4]:
# loading labels from csv file
df = pd.read_csv("Datasets/labels.csv")

# create dictionary of unique breeds with its respective id
breeds = df.breed.unique()
dict_breeds = dict(zip(breeds, range(len(breeds))))

# add duplicate column
df["breed_id"] = df.breed
# convert duplicated column as unique id
df = df.replace({"breed_id":dict_breeds})

print("Shape\t\t: {}".format(df.shape))
print("Unique labels\t: {}".format(len(breeds)))

Shape		: (10222, 3)
Unique labels	: 120


In [5]:
# hyperparameters 
EPOCHS = 50
LR = 1e-3
BATCH_SIZE = 32
INPUT_SHAPE = (64,64,3)

In [6]:
data = []
labels = []

# collect all files from directory into a list
image_files_train = [f for f in glob.glob("Datasets/train" + "/**/*", recursive=True) if not os.path.isdir(f)]
print("{} files found!".format(len(image_files_train)))

# create groud-truth label from the image path
print("loading images")
t = time.time()
for img in image_files_train:
    img_file = os.path.basename(img)
    name = img_file.split(".")[0]

    # check if image file has a record in given labels
    result = df.loc[df['id'] == name]
    if result.empty:
        print("LABEL NOT FOUND: {}".format(name))
        continue
    else:
        # reading of image 
        image = cv2.imread(img)
        image = cv2.resize(image, (INPUT_SHAPE[0], INPUT_SHAPE[1]))
        image = img_to_array(image)
        data.append(image)
        # read respective unique breed id from result 
        label = result['breed_id'].iloc[0]
        labels.append([label])
print(f'Time taken to load images: {time.time()-t}')

# pre-processing (normalisation)
print("pre-processing")
t = time.time()
data = np.array(data, dtype=np.float16) / 255.0
labels = np.array(labels, dtype=np.uint8)
print(f'Time taken to pre-processing: {time.time()-t}')

print('x_data shape:', data.shape)
print('y_data shape:', labels.shape)

10222 files found!
loading images
Time taken to load images: 42.19877624511719
pre-processing
Time taken to pre-processing: 1.398768663406372
x_data shape: (10222, 64, 64, 3)
y_data shape: (10222, 1)


In [7]:
def convert_size(size_bytes):
    if size_bytes == 0:
        return "0B"
    size_name = ("B", "KB", "MB", "GB", "TB", "PB", "EB", "ZB", "YB")
    i = int(math.floor(math.log(size_bytes, 1024)))
    p = math.pow(1024, i)
    s = round(size_bytes / p, 2)
    return "%s %s" % (s, size_name[i])

def nparray_info(header, nparr):
    print(header)
    print("-  shape\t", nparr.shape)
    print("-  dtype\t", nparr.dtype)
    print("- nbytes\t", f"{nparr.nbytes} ({convert_size(nparr.nbytes)})")
    

In [8]:
# print out shape, dtype and data size
nparray_info("Images", data)
print()
nparray_info("Labels", labels)

Images
-  shape	 (10222, 64, 64, 3)
-  dtype	 float16
- nbytes	 251215872 (239.58 MB)

Labels
-  shape	 (10222, 1)
-  dtype	 uint8
- nbytes	 10222 (9.98 KB)


### loading and saving with .npz
* [np.save()](https://numpy.org/doc/stable/reference/generated/numpy.save.html)
* [np.savez()](https://numpy.org/doc/stable/reference/generated/numpy.savez.html)
* [np.savez_compressed()](https://numpy.org/doc/stable/reference/generated/numpy.savez_compressed.html)
* [np.lib.format](https://numpy.org/doc/stable/reference/generated/numpy.lib.format.html)





In [9]:
saving_path = "./Datasets/preprocessed_data_{}x{}.npz".format(INPUT_SHAPE[0], INPUT_SHAPE[1])

print("Loading saving to npz file")
# ensure director is created before save data file
os.makedirs(os.path.dirname(saving_path), exist_ok=True)
t = time.time()
np.savez_compressed(saving_path, X=data, Y=labels)
print(f'Time taken to save compressed data: {time.time()-t}')

Loading saving to npz file
Time taken to save compressed data: 23.279183864593506


In [10]:
print("Loading from npz file")
t = time.time()
loaded = np.load(saving_path)
print(loaded.files)
loaded_X = loaded["X"]
loaded_Y = loaded["Y"]
print(f'Time taken to load compressed data: {time.time()-t}')

Loading from npz file
['X', 'Y']
Time taken to load compressed data: 1.7265007495880127


In [11]:
# print out shape, dtype and data size
nparray_info("Images (npz)", loaded_X)
print()
nparray_info("Labels (npz)", loaded_Y)

Images (npz)
-  shape	 (10222, 64, 64, 3)
-  dtype	 float16
- nbytes	 251215872 (239.58 MB)

Labels (npz)
-  shape	 (10222, 1)
-  dtype	 uint8
- nbytes	 10222 (9.98 KB)


In [12]:
# ensure correctness of data
assert labels.dtype == loaded_Y.dtype
assert (labels==loaded_Y).all()


assert data.dtype == loaded_X.dtype
assert np.allclose(data, loaded_X)
assert (data==loaded_X).all()

### loading and saving with h5 format
* [h5py.creating-datasets()](https://docs.h5py.org/en/stable/high/dataset.html#creating-datasets)
* [lossless compression](https://docs.h5py.org/en/stable/high/dataset.html#lossless-compression-filters)



In [13]:
def generate_h5_data(data, labels, file_path="./Datasets/data.h5"):
    assert(type(file_path) is str)
    assert(len(data) == len(labels))
    
    os.makedirs(os.path.dirname(saving_path), exist_ok=True)
    try:
        with h5py.File(file_path, 'w') as h5:
            h5.create_dataset('image', data= data, compression="gzip", compression_opts=9)
            h5.create_dataset('label', data= labels)
    except Exception as e:
        print(e)
    

def load_h5_data(file_path):
    try:
        with h5py.File(file_path, "r") as h5:
            print("h5.keys", h5.keys())
            data = np.array(h5['image'][:])
            labels = np.array(h5['label'][:])
        print("The shape of x_field",data.shape)
        print("The shape of y_field",labels.shape)
    except Exception as e:
        print(e)
    return data, labels

In [14]:
saving_path = "./Datasets/preprocessed_data_{}x{}.h5".format(INPUT_SHAPE[0], INPUT_SHAPE[1])

t = time.time()
generate_h5_data(data, labels, saving_path)
print(f'Time taken to save compressed data: {time.time()-t}')

Time taken to save compressed data: 20.961398124694824


In [15]:
print("loading from h5 file")
t = time.time()
loaded_X, loaded_Y = load_h5_data(saving_path)
print(f'Time taken to load compressed data: {time.time()-t}')

loading from h5 file
h5.keys <KeysViewHDF5 ['image', 'label']>
The shape of x_field (10222, 64, 64, 3)
The shape of y_field (10222, 1)
Time taken to load compressed data: 2.5167882442474365


In [16]:
# print out shape, dtype and data size
nparray_info("Images (h5)", loaded_X)
print()
nparray_info("Labels (h5)", loaded_Y)

Images (h5)
-  shape	 (10222, 64, 64, 3)
-  dtype	 float16
- nbytes	 251215872 (239.58 MB)

Labels (h5)
-  shape	 (10222, 1)
-  dtype	 uint8
- nbytes	 10222 (9.98 KB)


In [17]:
# ensure correctness of data
assert labels.dtype == loaded_Y.dtype
assert (labels==loaded_Y).all()


assert data.dtype == loaded_X.dtype
assert np.allclose(data, loaded_X)
assert (data==loaded_X).all()